Notebook to generate a MinMaxScaler and Normalizer object on the entire data set, then pickle them for later use.

This does not need to be run every time, just when we need to update the MinMaxScaler objects and Normalizer objects in the models folder because we've added a feature or something.

In [1]:
%run Preprocessing.ipynb

In [2]:
chunksize = 10 ** 6
filename = "../data/train.csv"

dataset_df = pd.DataFrame()  # main dataset

from tqdm import tqdm
# Pretty progress bar
pbar = tqdm(total=len(range(54)))       
for index, df_chunk in enumerate(pd.read_csv(filename, chunksize=chunksize)):
    pbar.update(1)
    dataset_df = pd.concat([dataset_df, preprocess(df_chunk)])
pbar.close()

size_bytes = dataset_df.memory_usage(deep=True).sum()
print("Entire dataset fits in {:.2f} GB".format(size_bytes/1e+9))
print("Number of rows: {}".format(dataset_df.count()[0]))

100%|██████████| 54/54 [02:41<00:00,  3.10s/it]


Entire dataset fits in 1.48 GB
Number of rows: 52756015


In [3]:
fares = dataset_df["fare_amount"]
del dataset_df["fare_amount"]
from sklearn.externals import joblib

In [4]:
dataset_df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,17,-73.844315,40.721317,-73.841614,40.712276,1
1,16,-74.016045,40.711304,-73.979271,40.782005,1
2,0,-73.982735,40.761269,-73.991241,40.750561,2
3,4,-73.987129,40.733143,-73.991570,40.758091,1
4,7,-73.968094,40.768009,-73.956657,40.783764,1


In [6]:
#Note may not overwrite old pickled files as we dump them, but append instead
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(dataset_df)
# then on new data call scaler.transform(data) to scale
with open('../models/minmaxscaler.joblib', 'wb') as file:
    joblib.dump(scaler, file) 
del scaler

/home/jovyan/.conda/envs/jovyan/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int8, float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [8]:
#  We need to normalize on scaled data I think, but I cant fit the scaled df into memory along with the original df
from sklearn.preprocessing import Normalizer
normalizer = Normalizer().fit(dataset_df)
with open('../models/normalizer.joblib', 'wb') as file:
    joblib.dump(normalizer, file) 
del normalizer